In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/hotel-id-2021-fgvc8/train.csv')
train_df

In [ ]:
train_df['/chain/image'] = train_df['chain'].astype(str) + '/' + train_df['image']
train_df

In [ ]:
#Filter Folder and Images to use in a new Dataframe
modelName='Equal90v2' #For Pickle File Name
train_df_subset=train_df.loc[train_df['chain']==90]
train_df_subset['hotel_id']=train_df_subset['hotel_id'].astype(str)
category_count=train_df_subset['hotel_id'].nunique()
train_df_subset.head(5)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Flow from Dataframe looks for image in filepath + x_col
filepath='../input/hotel-id-2021-fgvc8/train_images/'
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

training=datagen.flow_from_dataframe(
    dataframe=train_df_subset,
    directory=filepath,
    x_col="/chain/image",
    y_col="hotel_id",
    weight_col=None,
    #target_size=(256, 256),
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset='training',
    interpolation="nearest",
    validate_filenames=True
)

testing=datagen.flow_from_dataframe(
    dataframe=train_df_subset,
    directory=filepath,
    x_col="/chain/image",
    y_col="hotel_id",
    weight_col=None,
    #target_size=(256, 256),
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset='validation',
    interpolation="nearest",
    validate_filenames=True
)

In [ ]:
import tensorflow as tf
import numpy as np


try:
    model = tf.keras.models.load_model('./'+modelName)

except:
    model = tf.keras.applications.EfficientNetB0(
        include_top=True,
        weights=None,
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=category_count,
        classifier_activation="softmax"
    )
    
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    print('done model generation')

In [ ]:
model.fit(training)

In [ ]:
model.evaluate(testing)

In [ ]:
model.save(modelName)

In [ ]:
# f = open("listouput.txt",mode='x')
# f.write(hotelLabels)
# f.close()

import pickle

with open('listOutput.pickel', 'x') as f:
    pickle.dump(hotelLabels, f)